Load two surfaces $f_1$ and $f_2$ of size $3*N_{\phi}*N_{\theta}$, where $N_\phi$ denotes the number of discrete polar angle $\phi\in[0,\pi]$ and $N_\theta$ denotes the number of discrete azimuthal angle $\theta\in[0,2\pi]$.

To use the code, $N_{\theta}$ must be **odd** and the matrix representation of each surface should satisfy $f[:, 0, :] = f[:, 0, 0]$, $f[:, -1, :] = f[:, -1, 0]$ and $f[:, :, 0] = f[:, :, -1]$, i.e., $f(0, \theta) = f(0, 0)$, $f(\pi, \theta) = f(\pi, 0)$ and $f(\phi, 0) = f(\phi, 2\pi)$.

In [ ]:
import scipy.io as sio
from Packages.RegisterSurfaces import *

#  
mat_f= sio.loadmat('ShapeData/bumpSphere.mat')
# mat_f= sio.loadmat('ShapeData/TwoBumpSphere.mat')

f1 = torch.from_numpy(mat_f['f1']).double()
f2 = torch.from_numpy(mat_f['f2']).double()

In [ ]:
MaxDegVecFS2 = 2  # the maximal degree of spherical harmonics for the tangent vector fields on S2
a,b,c = 1, 1, 1  # choices of constants for the general elastic metric

# Initialize over the icosahedral group and then over the group of diffeomorphisms of rotations
f1_gamma, ESO3, f1_barh, EIco = initialize_over_paraSO3(f1, f2, a, b, c)

In [ ]:
%%time 
# minimize over the whole group of diffeomorphisms
# set the maximum number of iterations for the whole optimization process and the maximal number of iterations
# for each optimization 

numIte = (1,50)  
f, D = opt_overDiff(f1_gamma, f2, a, b, c, MaxDegVecFS2, numIte)

In [ ]:
# plot the energy 
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(D)
plt.show()

In [ ]:
# plot the registration result

idty = get_idty_S2(*f1.shape[-2:])

from mayavi import mlab
Gx, Gy = np.gradient(idty[2])  
Grad = (Gx**2+Gy**2)**.5 
w = Grad/Grad.max()  

%gui qt

mlab.init_notebook('x3d',1000,500) # png & x3d
mlab.figure(1, size=(100,70), fgcolor=(1, 1, 1), bgcolor=(0.5, 0.5, 0.5))
mlab.clf()

s1 = mlab.mesh(f1[0].numpy()-2, f1[1].numpy(), f1[2].numpy(),representation='wireframe',scalars=w)
s = mlab.mesh(f[0].numpy(), f[1].numpy(), f[2].numpy(),representation='wireframe',scalars=w)
s2 = mlab.mesh(f2[0].numpy()+2, f2[1].numpy(), f2[2].numpy(),representation='wireframe',scalars=w)


mlab.view(azimuth=270, elevation=90)
mlab.show()
s1